<a href="https://colab.research.google.com/github/TanimotoRui/signate/blob/main/17_lgb_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ライブラリのインポート / データの読み込み

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

%matplotlib inline

from matplotlib import pyplot as plt
import matplotlib.ticker as mtick # For specifying the axes tick format 

import seaborn as sns
import re

import json, os, gc, math, time
import datetime
import collections
from tqdm import tqdm
import glob

from statistics import mean
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

from sklearn import metrics
import time

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/00_datascience/19_ufj_bank"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/00_datascience/19_ufj_bank


In [ ]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sub = pd.read_csv('input/sample_submit.csv')

# bertによる特徴抽出

In [ ]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [ ]:
for i in range(len(train)):
  train.loc[i, 'cleaned_text'] = remove_html(train.loc[i, 'html_content'])

for i in range(len(test)):
  test.loc[i, 'cleaned_text'] = remove_html(test.loc[i, 'html_content'])

In [ ]:
#すでに抽出済みの特徴量を使用
text_train_df = pd.read_csv('input/05_seq_train_df.csv')
text_train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
text_test_df = pd.read_csv('input/05_seq_test_df.csv')
text_test_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
merge_text = pd.concat([text_train_df, text_test_df], ignore_index=True)

# ラベルデータの前処理

In [ ]:
#学習しやすいようにtrain, testをマージ
merge_df = pd.concat([train, test], ignore_index=True)

In [ ]:
#goalカラムの数値化
for i in range(len(merge_df)):
  price_list = re.findall(r"\d+", merge_df.loc[i, 'goal'])
  merge_df.loc[i, 'goal_min'] = price_list[0]
  if len(price_list) == 1:
    merge_df.loc[i, 'goal_max'] = price_list[0]
  else:
    merge_df.loc[i, 'goal_max'] = price_list[1]

In [ ]:
#int型に変換
merge_df['goal_max'] = merge_df['goal_max'].astype(int)
merge_df['goal_min'] = merge_df['goal_min'].astype(int)

#1日あたりの目標金額の算出
merge_df['goal_per_day_max'] = merge_df['goal_max'] / merge_df['duration']
merge_df['goal_per_day_min'] = merge_df['goal_min'] / merge_df['duration']
#merge_df.head()

In [ ]:
# ワンホットエンコーディング

dummy_cols = ['goal',	'country', 'category1',	'category2']
from sklearn.preprocessing import OneHotEncoder

dummy_df = pd.get_dummies(merge_df[dummy_cols])

In [ ]:
merge_df_new = pd.merge(merge_df, dummy_df, left_index=True, right_index=True)

In [ ]:
#ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

le_cols = ['goal', 'country', 'category1', 'category2']

#カテゴリ変数をループしてlabel encoding
for c in le_cols:
  le = LabelEncoder()
  le.fit(merge_df_new[c])
  merge_df_new[c] = le.transform(merge_df_new[c])

# tfidfによる特徴抽出

In [ ]:
#tfidfによる特徴抽出
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
features = tv.fit_transform(merge_df_new["cleaned_text"].fillna(""))

In [ ]:
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline



tfidf_svd = Pipeline(steps=[
    ("TfidfVectorizer", TfidfVectorizer()),
    ("TruncatedSVD", TruncatedSVD(n_components=100, random_state=42))
])

features_svd = tfidf_svd.fit_transform(merge_df_new["cleaned_text"].fillna(""))

In [ ]:
svd_df_merge = pd.DataFrame(features_svd)
svd_df_merge.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.294975,0.000612,0.023093,-0.043301,-0.004650,0.020541,-0.045336,0.023750,-0.013548,0.034169,-0.013015,0.006802,-0.034911,0.006794,-0.015047,0.005609,-0.006645,0.028476,0.020401,-0.021404,0.013798,0.013175,-0.021458,0.007860,-0.033133,0.021228,-0.028845,0.018220,0.040855,0.015536,-0.009352,-0.025442,-0.007781,0.021415,-0.003995,-0.002129,0.006594,-0.018173,-0.025319,-0.037783,0.027987,-0.042046,-0.017961,0.007527,-0.014963,0.028129,-0.023459,-0.019673,-0.003277,-0.039588,0.015771,-0.002947,0.044751,-0.010050,-0.024048,-0.011614,-0.023430,-0.030329,-0.001305,0.012928,0.024367,0.003222,0.029102,0.000107,0.024981,-0.015438,-0.007298,-0.024717,-0.015880,-0.007488,-0.009271,0.007005,0.014566,-0.031326,0.013497,-0.012960,-0.012240,-0.031298,-0.022263,0.014773,-0.003791,-0.008527,-0.014285,-0.000027,-0.010357,0.023047,-0.011880,-0.016816,-0.013937,-0.008330,0.016746,-0.007637,-0.006122,-0.005141,0.014197,-0.002046,0.005369,-0.004800,-0.021516,0.000758
1,0.163203,-0.003916,0.027252,0.018163,-0.002540,-0.027746,0.007868,0.058627,-0.076373,0.005526,0.001523,0.063076,0.065007,-0.036947,0.008218,-0.003941,0.084550,-0.047071,-0.007865,-0.024665,-0.022845,0.037984,0.000789,-0.017015,0.020834,-0.002827,0.013883,-0.020393,-0.010918,-0.013796,0.024166,0.002487,-0.000861,-0.031051,-0.004367,-0.018240,0.003508,-0.013320,-0.001160,0.019565,-0.003766,0.006770,0.006800,0.015419,-0.015384,0.009552,-0.021994,0.022982,0.007848,-0.000524,0.032245,-0.003441,0.021012,-0.015403,-0.023079,0.024831,-0.002129,-0.005817,-0.010186,0.009194,0.026070,0.012619,0.007577,0.002824,0.019720,-0.009614,0.014508,0.008810,0.012031,0.016641,-0.012554,-0.021404,-0.012819,0.021749,0.008886,-0.008196,0.005917,-0.008584,0.013794,-0.004741,-0.001814,-0.000274,0.018601,-0.005945,0.012709,-0.015027,0.023367,-0.002542,0.012418,-0.013684,0.009684,-0.030274,-0.001825,-0.025953,0.004676,-0.024310,-0.029303,0.023187,-0.023117,0.013129
2,0.416711,0.004678,-0.052108,-0.079082,-0.010637,0.082928,-0.018374,0.036624,-0.071622,-0.046464,-0.014945,0.041355,-0.020230,-0.010950,-0.087223,-0.007249,-0.030307,0.014107,-0.012528,-0.046732,-0.023661,0.035883,0.028221,-0.034219,-0.007161,-0.004547,0.046385,-0.009552,0.025520,-0.041551,0.013066,0.021094,0.029878,0.006288,-0.000849,-0.046704,-0.016616,-0.026287,-0.004538,0.004160,0.008717,0.002065,-0.044870,0.015324,0.036764,-0.038749,0.029809,-0.019620,0.012642,0.052022,0.012008,0.030016,0.046770,-0.004494,-0.028914,0.046057,0.028594,-0.005432,0.041683,-0.043660,-0.016415,-0.003372,0.040315,-0.002028,0.003212,0.026453,0.002681,0.016203,0.004950,-0.034195,0.011770,0.008469,-0.004297,0.001236,-0.025254,0.011300,-0.022994,0.021760,-0.022508,0.020674,-0.015725,-0.012703,-0.039726,-0.029907,-0.009900,0.017603,0.020235,0.002021,-0.010242,0.013850,-0.022286,-0.033044,0.070526,-0.007964,0.013953,0.036927,0.002910,-0.010230,-0.004970,-0.028130
3,0.354831,0.040357,-0.074435,-0.139759,-0.016493,0.005695,-0.074027,-0.054699,-0.033433,0.030256,0.012643,-0.062602,0.008236,-0.000220,-0.032510,-0.007413,-0.049098,0.035602,-0.013406,-0.027928,0.020493,-0.020341,0.060540,-0.026912,-0.018913,-0.024155,0.034550,-0.004458,-0.079302,0.025177,-0.036539,-0.026917,-0.002154,-0.015356,-0.007210,0.007557,0.001892,0.008017,-0.002577,0.028486,-0.033584,0.013869,-0.032748,0.004899,0.018109,0.003867,0.003197,-0.002485,-0.019856,-0.002880,0.025240,0.004297,-0.009714,0.023688,0.011628,0.002504,0.000008,0.000278,-0.022154,-0.011611,0.005533,-0.038461,0.021524,-0.008633,0.003877,-0.047406,0.027908,0.000041,0.038034,0.019338,0.013657,0.014267,0.055440,-0.032712,-0.025378,-0.009006,-0.012042,-0.034948,0.000290,-0.041465,0.001809,0.006383,0.014765,0.025630,0.010260,-0.040052,-0.014977,-0.003671,0.

# w2vによる特徴抽出

https://qiita.com/propella/items/febc423998fd210800ca
よりフォーク

In [ ]:
#単語に対して分散表現を与える
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(merge_df_new["cleaned_text"])]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
#コサイン類似度が上位n個のものを抽出
n = 10
for i in range(len(merge_df_new)):
  print(i)
  for j in range(n):
    merge_df_new.loc[i, 'similar{}'.format(j)] = model.docvecs.most_similar(i)[j][0]

ストリーミング出力は最後の 5000 行に切り捨てられました。
14591
14592
14593
14594
14595
14596
14597
14598
14599
14600
14601
14602
14603
14604
14605
14606
14607
14608
14609
14610
14611
14612
14613
14614
14615
14616
14617
14618
14619
14620
14621
14622
14623
14624
14625
14626
14627
14628
14629
14630
14631
14632
14633
14634
14635
14636
14637
14638
14639
14640
14641
14642
14643
14644
14645
14646
14647
14648
14649
14650
14651
14652
14653
14654
14655
14656
14657
14658
14659
14660
14661
14662
14663
14664
14665
14666
14667
14668
14669
14670
14671
14672
14673
14674
14675
14676
14677
14678
14679
14680
14681
14682
14683
14684
14685
14686
14687
14688
14689
14690
14691
14692
14693
14694
14695
14696
14697
14698
14699
14700
14701
14702
14703
14704
14705
14706
14707
14708
14709
14710
14711
14712
14713
14714
14715
14716
14717
14718
14719
14720
14721
14722
14723
14724
14725
14726
14727
14728
14729
14730
14731
14732
14733
14734
14735
14736
14737
14738
14739
14740
14741
14742
14743
14744
14745
14746
14747
14748
14749
14750
14751
14

In [ ]:
merge_df_new.head()

,id,goal,country,duration,category1,category2,html_content,state,cleaned_text,goal_min,goal_max,goal_per_day_max,goal_per_day_min,goal_1-1000,goal_100000+,goal_10001-11000,goal_1001-2000,goal_11001-12000,goal_12001-13000,goal_13001-14000,goal_14001-15000,goal_15001-16000,goal_16001-17000,goal_17001-18000,goal_18001-19000,goal_19001-20000,goal_20001-21000,goal_2001-3000,goal_21001-22000,goal_22001-23000,goal_23001-24000,goal_24001-25000,goal_25001-26000,goal_26001-27000,goal_27001-28000,goal_28001-29000,goal_29001-30000,goal_30001-31000,goal_3001-4000,goal_31001-32000,goal_32001-33000,goal_33001-34000,goal_34001-35000,goal_35001-36000,goal_36001-37000,goal_37001-38000,goal_38001-39000,goal_39001-40000,goal_40001-41000,goal_4001-5000,goal_41001-42000,goal_42001-43000,goal_43001-44000,goal_44001-45000,goal_45001-46000,goal_46001-47000,goal_47001-48000,goal_48001-49000,goal_49001-50000,goal_50001-51000,goal_5001-6000,goal_51001-52000,goal_52001-53000,goal_53001-54000,goal_54001-55000,goal_55001-56000,goal_56001-57000,goal_57001-58000,goal_58001-59000,goal_59001-60000,goal_60001-61000,goal_6001-7000,goal_61001-62000,goal_62001-63000,goal_63001-64000,goal_64001-65000,goal_65001-66000,goal_66001-67000,goal_67001-68000,goal_68001-69000,goal_69001-70000,goal_70001-71000,goal_7001-8000,goal_71001-72000,goal_72001-73000,goal_73001-74000,goal_74001-75000,goal_75001-76000,goal_76001-77000,goal_77001-78000,goal_78001-79000,goal_79001-80000,goal_80001-81000,goal_8001-9000,goal_81001-82000,goal_82001-83000,goal_83001-84000,goal_84001-85000,goal_85001-86000,goal_86001-87000,goal_87001-88000,goal_88001-89000,goal_89001-90000,goal_90001-91000,goal_9001-10000,goal_91001-92000,goal_92001-93000,goal_93001-94000,goal_94001-95000,goal_95001-96000,goal_96001-97000,goal_97001-98000,goal_98001-99000,goal_99001-100000,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,category1_art,category1_comics,category1_crafts,category1_dance,category1_design,category1_fashion,category1_film & video,category1_food,category1_games,category1_journalism,category1_music,category1_photography,category1_publishing,category1_technology,category1_theater,category2_3d printing,category2_academic,category2_accessories,category2_action,category2_animals,category2_animation,category2_anthologies,category2_apparel,category2_apps,category2_architecture,category2_art books,category2_audio,category2_bacon,category2_blues,category2_calendars,category2_camera equipment,category2_candles,category2_ceramics,category2_children's books,category2_childrenswear,category2_chiptune,category2_civic design,category2_classical music,category2_comedy,category2_comic books,category2_community gardens,category2_conceptual art,category2_cookbooks,category2_country & folk,category2_couture,category2_crochet,category2_digital art,category2_diy,category2_diy electronics,category2_documentary,category2_drama,category2_drinks,category2_electronic music,category2_embroidery,category2_events,category2_experimental,category2_fabrication tools,category2_faith,category2_family,category2_fantasy,category2_farmer's markets,category2_farms,category2_festivals,category2_fiction,category2_fine art,category2_flight,category2_food trucks,category2_footwear,category2_gadgets,category2_gaming hardware,category2_glass,category2_graphic design,category2_graphic novels,category2_hardware,category2_hip-hop,category2_horror,category2_illustration,category2_immersive,category2_indie rock,category2_installations,category2_interactive design,category2_jazz,category2_jewelry,category2_kids,category2_knitting,category2_latin,category2_letterpress,category2_literary journals,category2_literary spaces,category2_live games,category2_makerspaces,category2_metal,category2_mixed media,category2_mobile games,category2_movie theaters,category2_mus

# データのマージ/kmeans用の前処理

In [ ]:
#不要なカラムの定義
non_use_cols = ['id',	'html_content',	'cleaned_text', 'state']

#スケーリングするカラムの選択
sc_cols = ['duration', 'goal_max', 'goal_min', 'goal_per_day_max', 'goal_per_day_min']

In [ ]:
#bertから抽出した特徴量を追加
full_merge_df = pd.merge(merge_df_new.drop(non_use_cols, axis=1), merge_text, left_index=True, right_index=True)

In [ ]:
#tfidfから抽出した特徴量を追加
full_merge_df_fin = pd.merge(full_merge_df, svd_df_merge, left_index=True, right_index=True)

In [ ]:
full_merge_df_fin.head()

,goal,country,duration,category1,category2,goal_min,goal_max,goal_per_day_max,goal_per_day_min,goal_1-1000,goal_100000+,goal_10001-11000,goal_1001-2000,goal_11001-12000,goal_12001-13000,goal_13001-14000,goal_14001-15000,goal_15001-16000,goal_16001-17000,goal_17001-18000,goal_18001-19000,goal_19001-20000,goal_20001-21000,goal_2001-3000,goal_21001-22000,goal_22001-23000,goal_23001-24000,goal_24001-25000,goal_25001-26000,goal_26001-27000,goal_27001-28000,goal_28001-29000,goal_29001-30000,goal_30001-31000,goal_3001-4000,goal_31001-32000,goal_32001-33000,goal_33001-34000,goal_34001-35000,goal_35001-36000,goal_36001-37000,goal_37001-38000,goal_38001-39000,goal_39001-40000,goal_40001-41000,goal_4001-5000,goal_41001-42000,goal_42001-43000,goal_43001-44000,goal_44001-45000,goal_45001-46000,goal_46001-47000,goal_47001-48000,goal_48001-49000,goal_49001-50000,goal_50001-51000,goal_5001-6000,goal_51001-52000,goal_52001-53000,goal_53001-54000,goal_54001-55000,goal_55001-56000,goal_56001-57000,goal_57001-58000,goal_58001-59000,goal_59001-60000,goal_60001-61000,goal_6001-7000,goal_61001-62000,goal_62001-63000,goal_63001-64000,goal_64001-65000,goal_65001-66000,goal_66001-67000,goal_67001-68000,goal_68001-69000,goal_69001-70000,goal_70001-71000,goal_7001-8000,goal_71001-72000,goal_72001-73000,goal_73001-74000,goal_74001-75000,goal_75001-76000,goal_76001-77000,goal_77001-78000,goal_78001-79000,goal_79001-80000,goal_80001-81000,goal_8001-9000,goal_81001-82000,goal_82001-83000,goal_83001-84000,goal_84001-85000,goal_85001-86000,goal_86001-87000,goal_87001-88000,goal_88001-89000,goal_89001-90000,goal_90001-91000,goal_9001-10000,goal_91001-92000,goal_92001-93000,goal_93001-94000,goal_94001-95000,goal_95001-96000,goal_96001-97000,goal_97001-98000,goal_98001-99000,goal_99001-100000,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,category1_art,category1_comics,category1_crafts,category1_dance,category1_design,category1_fashion,category1_film & video,category1_food,category1_games,category1_journalism,category1_music,category1_photography,category1_publishing,category1_technology,category1_theater,category2_3d printing,category2_academic,category2_accessories,category2_action,category2_animals,category2_animation,category2_anthologies,category2_apparel,category2_apps,category2_architecture,category2_art books,category2_audio,category2_bacon,category2_blues,category2_calendars,category2_camera equipment,category2_candles,category2_ceramics,category2_children's books,category2_childrenswear,category2_chiptune,category2_civic design,category2_classical music,category2_comedy,category2_comic books,category2_community gardens,category2_conceptual art,category2_cookbooks,category2_country & folk,category2_couture,category2_crochet,category2_digital art,category2_diy,category2_diy electronics,category2_documentary,category2_drama,category2_drinks,category2_electronic music,category2_embroidery,category2_events,category2_experimental,category2_fabrication tools,category2_faith,category2_family,category2_fantasy,category2_farmer's markets,category2_farms,category2_festivals,category2_fiction,category2_fine art,category2_flight,category2_food trucks,category2_footwear,category2_gadgets,category2_gaming hardware,category2_glass,category2_graphic design,category2_graphic novels,category2_hardware,category2_hip-hop,category2_horror,category2_illustration,category2_immersive,category2_indie rock,category2_installations,category2_interactive design,category2_jazz,category2_jewelry,category2_kids,category2_knitting,category2_latin,category2_letterpress,category2_literary journals,category2_literary spaces,category2_live games,category2_makerspaces,category2_metal,category2_mixed media,category2_mobile games,category2_movie theaters,category2_music videos,category2_musical,categor

In [ ]:
#sc_colsのスケーリング
from sklearn.preprocessing import StandardScaler

for c in sc_cols:
  std = StandardScaler()
  full_merge_df_fin[c] = std.fit_transform(pd.DataFrame(full_merge_df_fin[c]))

# kmeansによるクラスタリング

In [ ]:
from sklearn.cluster import KMeans
non_use_cols_for_km = [
    'goal', 
    'country', 
    'category1', 
    'category2', 
    'similar0', 
    'similar1', 
    'similar2', 
    'similar3', 
    'similar4', 
    'similar5', 
    'similar6', 
    'similar7', 
    'similar8', 
    'similar9']

In [ ]:
# エルボー方による推定。クラスター数を1から20に増やして、それぞれの距離の総和を求める
"""
dist_list =[]
for i in range(1,50):
    print('今はクラスター数{}のクラスタリングを実行しています。'.format(i))
    kmeans= KMeans(n_clusters=i, init='random', random_state=0)
    kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
    dist_list.append(kmeans.inertia_)
    
# グラフを表示
plt.plot(range(1,50), dist_list,marker='+')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
"""

"\ndist_list =[]\nfor i in range(1,50):\n    print('今はクラスター数{}のクラスタリングを実行しています。'.format(i))\n    kmeans= KMeans(n_clusters=i, init='random', random_state=0)\n    kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))\n    dist_list.append(kmeans.inertia_)\n    \n# グラフを表示\nplt.plot(range(1,50), dist_list,marker='+')\nplt.xlabel('Number of clusters')\nplt.ylabel('Distortion')\n"

In [ ]:
#クラスタリング
clusters_list=[5, 10, 15, 20]

for n_clusters in clusters_list:
  print('今クラスター数[{}]のクラスタリングをしています。'.format(n_clusters))
  kmeans = KMeans(init='random', n_clusters=n_clusters, random_state=0)
  kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
  full_merge_df_fin['cluster_number_{}'.format(n_clusters)] =pd.Series(kmeans.labels_, name='cluster_number_{}'.format(5))

今クラスター数[5]のクラスタリングをしています。
今クラスター数[10]のクラスタリングをしています。
今クラスター数[15]のクラスタリングをしています。
今クラスター数[20]のクラスタリングをしています。


# aggrigation特徴量の作成

In [ ]:
#aggrigationの際にgroup化するカラム
group_cols = ['goal',	'country', 'category1',	'category2', 'cluster_number_5', 'cluster_number_10', 'cluster_number_15', 'cluster_number_20', 'similar0', 'similar1', 'similar2', 'similar3', 'similar4', 'similar5', 'similar6', 'similar7', 'similar8', 'similar9']

In [ ]:
#関数定義
def agg_func_duration(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["duration"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_duration")

  return output_df

def agg_func_state(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["duration"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_state")

  return output_df

def agg_func_goal_per_day_max(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["goal_per_day_max"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_state")

  return output_df

def agg_func_goal_per_day_min(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["goal_per_day_min"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_state")

  return output_df

In [ ]:
#実行の準備
full_merge_df_fin["state"] = train['state']
agg_df = agg_func_duration(full_merge_df_fin, group_cols[0])

In [ ]:
#関数の実行
for c in group_cols[1:len(group_cols)]:
  agg_df = pd.merge(agg_df, agg_func_duration(full_merge_df_fin, c), left_index=True, right_index=True)

for c in group_cols:
  agg_df = pd.merge(agg_df, agg_func_state(full_merge_df_fin, c), left_index=True, right_index=True)

for c in group_cols:
  agg_df = pd.merge(agg_df, agg_func_goal_per_day_max(full_merge_df_fin, c), left_index=True, right_index=True)

for c in group_cols:
  agg_df = pd.merge(agg_df, agg_func_goal_per_day_min(full_merge_df_fin, c), left_index=True, right_index=True)

In [ ]:
agg_df

,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,m

In [ ]:
#データのマージ
full_merge_df_fin = pd.merge(full_merge_df_fin, agg_df, left_index=True, right_index=True)
full_merge_df_fin.drop(['state'], axis=1, inplace=True)

# データセットの整理

In [ ]:
train_df = pd.DataFrame(full_merge_df_fin.loc[:len(train)-1,])
test_df = pd.DataFrame(full_merge_df_fin.loc[len(train):19591,])

In [ ]:
test_df = test_df.reset_index()
test_df.drop(['index'], axis=1, inplace=True)
test_df.head()

,goal,country,duration,category1,category2,goal_min,goal_max,goal_per_day_max,goal_per_day_min,goal_1-1000,goal_100000+,goal_10001-11000,goal_1001-2000,goal_11001-12000,goal_12001-13000,goal_13001-14000,goal_14001-15000,goal_15001-16000,goal_16001-17000,goal_17001-18000,goal_18001-19000,goal_19001-20000,goal_20001-21000,goal_2001-3000,goal_21001-22000,goal_22001-23000,goal_23001-24000,goal_24001-25000,goal_25001-26000,goal_26001-27000,goal_27001-28000,goal_28001-29000,goal_29001-30000,goal_30001-31000,goal_3001-4000,goal_31001-32000,goal_32001-33000,goal_33001-34000,goal_34001-35000,goal_35001-36000,goal_36001-37000,goal_37001-38000,goal_38001-39000,goal_39001-40000,goal_40001-41000,goal_4001-5000,goal_41001-42000,goal_42001-43000,goal_43001-44000,goal_44001-45000,goal_45001-46000,goal_46001-47000,goal_47001-48000,goal_48001-49000,goal_49001-50000,goal_50001-51000,goal_5001-6000,goal_51001-52000,goal_52001-53000,goal_53001-54000,goal_54001-55000,goal_55001-56000,goal_56001-57000,goal_57001-58000,goal_58001-59000,goal_59001-60000,goal_60001-61000,goal_6001-7000,goal_61001-62000,goal_62001-63000,goal_63001-64000,goal_64001-65000,goal_65001-66000,goal_66001-67000,goal_67001-68000,goal_68001-69000,goal_69001-70000,goal_70001-71000,goal_7001-8000,goal_71001-72000,goal_72001-73000,goal_73001-74000,goal_74001-75000,goal_75001-76000,goal_76001-77000,goal_77001-78000,goal_78001-79000,goal_79001-80000,goal_80001-81000,goal_8001-9000,goal_81001-82000,goal_82001-83000,goal_83001-84000,goal_84001-85000,goal_85001-86000,goal_86001-87000,goal_87001-88000,goal_88001-89000,goal_89001-90000,goal_90001-91000,goal_9001-10000,goal_91001-92000,goal_92001-93000,goal_93001-94000,goal_94001-95000,goal_95001-96000,goal_96001-97000,goal_97001-98000,goal_98001-99000,goal_99001-100000,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,category1_art,category1_comics,category1_crafts,category1_dance,category1_design,category1_fashion,category1_film & video,category1_food,category1_games,category1_journalism,category1_music,category1_photography,category1_publishing,category1_technology,category1_theater,category2_3d printing,category2_academic,category2_accessories,category2_action,category2_animals,category2_animation,category2_anthologies,category2_apparel,category2_apps,category2_architecture,category2_art books,category2_audio,category2_bacon,category2_blues,category2_calendars,category2_camera equipment,category2_candles,category2_ceramics,category2_children's books,category2_childrenswear,category2_chiptune,category2_civic design,category2_classical music,category2_comedy,category2_comic books,category2_community gardens,category2_conceptual art,category2_cookbooks,category2_country & folk,category2_couture,category2_crochet,category2_digital art,category2_diy,category2_diy electronics,category2_documentary,category2_drama,category2_drinks,category2_electronic music,category2_embroidery,category2_events,category2_experimental,category2_fabrication tools,category2_faith,category2_family,category2_fantasy,category2_farmer's markets,category2_farms,category2_festivals,category2_fiction,category2_fine art,category2_flight,category2_food trucks,category2_footwear,category2_gadgets,category2_gaming hardware,category2_glass,category2_graphic design,category2_graphic novels,category2_hardware,category2_hip-hop,category2_horror,category2_illustration,category2_immersive,category2_indie rock,category2_installations,category2_interactive design,category2_jazz,category2_jewelry,category2_kids,category2_knitting,category2_latin,category2_letterpress,category2_literary journals,category2_literary spaces,category2_live games,category2_makerspaces,category2_metal,category2_mixed media,category2_mobile games,category2_movie theaters,category2_music videos,category2_musical,categor

# ハイパラの調整

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9791 entries, 0 to 9790
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            9791 non-null   object
 1   goal          9791 non-null   object
 2   country       9791 non-null   object
 3   duration      9791 non-null   int64 
 4   category1     9791 non-null   object
 5   category2     9791 non-null   object
 6   html_content  9791 non-null   object
 7   state         9791 non-null   int64 
 8   cleaned_text  9791 non-null   object
dtypes: int64(2), object(7)
memory usage: 688.6+ KB


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9791 entries, 0 to 9790
Columns: 1534 entries, goal to std_agg_func_state_y
dtypes: float64(1243), int32(4), int64(4), uint8(283)
memory usage: 95.9 MB


In [ ]:
from sklearn.model_selection import train_test_split

def objective(trial):
    train_x, test_x, train_y, test_y = train_test_split(train_df, train['state'], test_size=0.25, random_state=0, stratify=train['state'])
    dtrain = lgb.Dataset(train_x, label=train_y)
 
    param = {
        'objective': 'binary',
        'metric': 'logloss',
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
 
    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(test_y, pred_labels)
    return accuracy

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2022-09-12 14:16:25,201] A new study created in memory with name: no-name-8604d12c-3791-45e0-91c5-4a8d3fd132ca
[I 2022-09-12 14:16:30,948] Trial 0 finished with value: 0.7912581699346405 and parameters: {'max_depth': 3, 'lambda_l1': 9.644425933827309e-05, 'lambda_l2': 0.0005124688550371716, 'num_leaves': 122, 'feature_fraction': 0.8123492581591932, 'bagging_fraction': 0.5671002656745363, 'bagging_freq': 7, 'min_child_samples': 43}. Best is trial 0 with value: 0.7912581699346405.
[I 2022-09-12 14:16:50,821] Trial 1 finished with value: 0.8022875816993464 and parameters: {'max_depth': 8, 'lambda_l1': 3.109833389148149, 'lambda_l2': 6.330992829362742e-08, 'num_leaves': 252, 'feature_fraction': 0.44517196642004536, 'bagging_fraction': 0.9459149491560128, 'bagging_freq': 5, 'min_child_samples': 9}. Best is trial 1 with value: 0.8022875816993464.
[I 2022-09-12 14:17:10,025] Trial 2 finished with value: 0.7961601307189542 and parameters: {'max_depth': 9, 'lambda_l1': 0.0002258593557600649,

In [ ]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

Number of finished trials: 100
Best trial: {'max_depth': 7, 'lambda_l1': 0.000574426100948834, 'lambda_l2': 2.304131739420853e-07, 'num_leaves': 172, 'feature_fraction': 0.4351792178840505, 'bagging_fraction': 0.7222477701017944, 'bagging_freq': 5, 'min_child_samples': 88}


Number of finished trials: 100
Best trial: {

'max_depth': 7, 

'lambda_l1': 0.000574426100948834, 

'lambda_l2': 2.304131739420853e-07, 

'num_leaves': 172, 

'feature_fraction': 0.4351792178840505, 

'bagging_fraction': 0.7222477701017944, 

'bagging_freq': 5, 

'min_child_samples': 88}

# lgbによる予測

In [ ]:
params = study.best_trial.params

In [ ]:
target = train['state']

In [ ]:
# 3分割交差検証を指定し、インスタンス化
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=0)  # 5分割交差検証のためにインスタンス化
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# スコアとモデルを格納するリスト
score_list = []
models = []

print('input size is {}行, {}カラム'.format(len(train_df), len(train_df.columns)))
for fold_, (train_index, valid_index) in enumerate(skf.split(train, target)):
    train_x = train_df.iloc[train_index]
    valid_x = train_df.iloc[valid_index]
    train_y = target[train_index]
    valid_y = target[valid_index]
    
    print(f'fold{fold_ + 1} start')

    gbm = lgb.LGBMClassifier(
        objective='binary',
        num_boost_round=50000, 
        learning_rate=0.005,
        max_depth=7,
        lambda_l1 = 0.000574426100948834,
        lambda_l2 = 2.304131739420853e-07,
        num_leaves = 172,
        feature_fraction = 0.4351792178840505,
        bagging_fraction = 0.7222477701017944,
        bagging_freq = 5,
        min_child_samples = 88
        )
    gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
                early_stopping_rounds=100,
                verbose= 100) # 学習の状況を表示しない
    
    oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    models.append(gbm)  # 学習が終わったモデルをリストに入れておく
    print(f'fold{fold_ + 1} end\n' )
print(score_list, '平均score', np.mean(score_list), "%")  

input size is 9791行, 1534カラム
fold1 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.586686
[200]	valid_0's binary_logloss: 0.53127
[300]	valid_0's binary_logloss: 0.497455
[400]	valid_0's binary_logloss: 0.476276
[500]	valid_0's binary_logloss: 0.46126
[600]	valid_0's binary_logloss: 0.450839
[700]	valid_0's binary_logloss: 0.443566
[800]	valid_0's binary_logloss: 0.437904
[900]	valid_0's binary_logloss: 0.433129
[1000]	valid_0's binary_logloss: 0.429885
[1100]	valid_0's binary_logloss: 0.427212
[1200]	valid_0's binary_logloss: 0.424508
[1300]	valid_0's binary_logloss: 0.422463
[1400]	valid_0's binary_logloss: 0.421092
[1500]	valid_0's binary_logloss: 0.419622
[1600]	valid_0's binary_logloss: 0.418491
[1700]	valid_0's binary_logloss: 0.417365
[1800]	valid_0's binary_logloss: 0.416485
[1900]	valid_0's binary_logloss: 0.415578
[2000]	valid_0's binary_logloss: 0.414675
[2100]	valid_0's binary_logloss: 0.414383
[2200]	valid_0's binary_l

In [ ]:
# testの予測
test_pred = np.zeros((len(test), 5))  # 行:len(test), 列:3のall zeroの配列を用意

for fold_, gbm in enumerate(models):  # 学習ずみのmodelをgbmに入れる
    pred_ = gbm.predict(test_df, num_iteration=gbm.best_iteration_)  # testの予測
    test_pred[:, fold_] = pred_  # １回目は0列目、2回目は1列目、2回目は3列目に格納

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)  # 平均をとって、0と１に変換

# submission

In [ ]:
sub = pd.DataFrame(test['id'])

In [ ]:
sub['state'] = pred
sub.head()

,id,state
0,test_00000,1
1,test_00001,1
2,test_00002,1
3,test_00003,0
4,test_00004,0


In [ ]:
sub.to_csv('sub/17_lgb_tuning_lr0.005.csv',index=False, header=None)